In [1]:
!pip -q install gradio crewai crewai-tools google-generativeai pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 576.3/576.3 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.0/755.0 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.9/323.9 kB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.9/19.9 MB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.9/157.9 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.7/38.7 MB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 4.8 MB

In [2]:
!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.3 MB/s eta 0:00:00


In [3]:
import os
import shutil
import zipfile
import tempfile
import gradio as gr
import google.generativeai as genai
from PyPDF2 import PdfReader
from crewai import Agent, Task, Crew, Process
from crewai import LLM

In [4]:
import os
import google.generativeai as genai
from google.colab import userdata
from google.colab import files
# Set your API key here
os.environ["GOOGLE_API_KEY"] = userdata.get("gemini_api2")

# genai.configure(api_key=os.environ["GOOGLE_API_KEY"])
gemini_api_key=userdata.get("gemini_api2")

In [5]:
from crewai import LLM
# llm = LLM(model="gemini/gemini-1.5-flash", api_key=gemini_api_key)
llm = LLM(model="gemini/gemini-2.5-flash", api_key=gemini_api_key)

In [6]:
import re

In [10]:
import os
from crewai import Agent, Task, Crew, LLM
def generate_portfolio():
  print("📄 Upload your resume PDF:")
  pdf_uploaded = files.upload()

  print("🖼️ Upload your profile photo (JPG/PNG):")
  img_uploaded = files.upload()

  # Extract file names
  pdf_path = list(pdf_uploaded.keys())[0]
  photo_filename = list(img_uploaded.keys())[0]
  profile_text = extract_text(pdf_path)


  extractor = Agent(
      role="Resume Extractor",
      goal="Extract professional details from resumes",
      backstory="Expert at extracting clean, structured data from unstructured PDFs.",
      llm=llm,
      verbose=True
  )

  builder = Agent(
      role="Portfolio Website Generator",
      goal="Create responsive HTML/CSS portfolio websites with user-uploaded photos",
      backstory="Frontend developer who always uses local image files correctly (e.g., photo.jpg)",
      llm=llm,
      verbose=True
  )
  extract_task = Task(
      description=f"""Extract structured information like name, summary, experience, education, and skills
      from this resume:\n\n{profile_text}""",
      expected_output="A structured resume summary in sections (name, title, skills, experience, education, etc.)",
      agent=extractor
  )

  builder_task = Task(
      description=f"""Using the structured resume content from the previous task, generate a clean and responsive portfolio website.

  - The user's profile image has been uploaded as `{photo_filename}`
  - Use this image in your HTML via: <img src="{photo_filename}" alt="Profile photo">
  - Do NOT use placeholders or external URLs
  -at the footer of the project add that this thing  {"this portfolio is generated by rohini sankari's generative ai project-ResumeToPortfolio"} and anchor to{"https://github.com/RohiniShankari/ResumeToPortfolio"}
  Output two code blocks:
  - One for `index.html`
  - One for `style.css`
  """,
      expected_output="Two full code blocks: one for HTML, one for CSS. HTML should use the uploaded photo.",
      agent=builder,
      depends_on=[extract_task]
  )

  crew = Crew(
    agents=[extractor, builder],
    tasks=[extract_task, builder_task],
    verbose=True
  )

  result = crew.kickoff()

  # Get final raw output
  final_output = result.tasks_output[-1].raw
  os.makedirs("portfolio", exist_ok=True)

  html_match = re.search(r"```html(.*?)```", final_output, re.DOTALL)
  css_match = re.search(r"```css(.*?)```", final_output, re.DOTALL)

  html_code = html_match.group(1).strip() if html_match else "<!-- HTML not found -->"
  css_code = css_match.group(1).strip() if css_match else "/* CSS not found */"

  with open("portfolio/index.html", "w") as f:
      f.write(html_code)

  with open("portfolio/style.css", "w") as f:
      f.write(css_code)

  zip_path = "portfolio.zip"
  photo_path = f"portfolio/{photo_filename}"

  # Make sure the image is in the folder
  if not os.path.exists(photo_path):
      import shutil
      shutil.copy(photo_filename, photo_path)

  # Create ZIP with all contents of portfolio/
  with zipfile.ZipFile(zip_path, 'w') as zipf:
      for root, dirs, files_in_dir in os.walk("portfolio"):
          for file in files_in_dir:
              full_path = os.path.join(root, file)
              arcname = os.path.relpath(full_path, "portfolio")  # keeps file names clean in ZIP
              zipf.write(full_path, arcname)

  # Show what's in the zip (for debug)
  print("📦 Files included in ZIP:")
  with zipfile.ZipFile(zip_path, 'r') as z:
      print(z.namelist())

  # Trigger download
  files.download(zip_path)

In [11]:
from PyPDF2 import PdfReader

def extract_text(pdf_path):
    reader = PdfReader(pdf_path)
    text = ""
    for page in reader.pages:
        text += page.extract_text() + "\n"
    return text


In [12]:
generate_portfolio()


📄 Upload your resume PDF:


Saving RESUME3 (16).pdf to RESUME3 (16) (1).pdf
🖼️ Upload your profile photo (JPG/PNG):


Saving WhatsApp Image 2025-10-13 at 11.07.46.jpeg to WhatsApp Image 2025-10-13 at 11.07.46 (1).jpeg


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 32a211d1-fd41-49f0-aaaf-c298b933d7be                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Resume Extractor                                                                                        │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  ```json                                                                                                        │
│  {                                                                                                              │
│    "name": "ROHINI SANKARI BYRA",                                                                               │
│    "contact": {                                                                                                 │
│      "email": "byrarohinisankari@gmail.com",                                                                    │
│      "phone": "(+91) 8499084617",                                                                               │
│      "github": "https://github.com/RohiniShankari",                                                             │
│      "linkedin": "https://www.linkedin.com/in/rohini-sankari-byra-32252b274/"                                   │
│    },                                                                                                           │
│    "summary": null,                                                                                             │
│    "skills": {                                                                                                  │
│      "programming_languages": [                                                                                 │
│        "Java",                                                                                                  │
│        "Python",                                                                                                │
│        "C"                                                                                                      │
│      ],                                                                                                         │
│      "web_development": [                                                                                       │
│        "HTML",                                                                                                  │
│        "CSS",                                                                                                   │
│        "JavaScript",                                                                                            │
│        "React",                                                                                                 │
│        "Node.js",                                                                                               │
│        "Express.js",                                                                                            │
│        "Bootstrap"                                                                                              │
│      ],                                                                                                         │
│      "databases": [                                                                                             │
│        "SQL",                                                                                                   │
│        "MongoDB"                                                                                                │
│      ],                                                                                                         │
│      "tools_platforms": [                              

Output()